In [1]:
from src.densite_function import *
from src.gaussian_simulation import *
from src.estimators import *
from src.vraisemblance import *
import numpy as np
from numpy.linalg import norm 
import matplotlib.pylab as plt

In [2]:
np.random.seed(8878)

theta=simulate_gaussian_vector(mu=np.array([0]*20), sigma=np.identity(20))

A_optimal=np.matrix(np.identity(20))*0.5
b_optimal=theta/2

A=A_optimal+simulate_gaussian_vector(mu=np.array([0]*20), sigma=0.01*np.identity(20))
b=b_optimal+simulate_gaussian_vector(mu=np.array([0]*20), sigma=0.01*np.identity(20))

On généré un n-échantillon de taille 100, de loi $p_{\theta}(\mathbf{x})=\mathcal{N}(\theta, 2I_{20})$

In [3]:
np.random.seed(8878)

def generer_nech_gaussien(n):
    i=1
    echantillon_x=np.array([])

    while i<=n:
        if i==1:
            echantillon_x=np.append(echantillon_x, simulate_gaussian_vector(mu=theta, sigma=2*np.identity(20)))
        else:
            echantillon_x=np.vstack((echantillon_x, simulate_gaussian_vector(mu=theta, sigma=2*np.identity(20))))
        i+=1
    return echantillon_x

echantillon_x=generer_nech_gaussien(100)

La log-vraisemblance de l'échantillon (avec le facteur $-\frac{1}{n}$) est $l_n(\theta)=\frac{1}{n}\sum_{i=1}^n \lVert X_i - \theta \rVert^2 = \frac{1}{n}\sum_{i=1}^n l_i(\theta) $. 
On cherche $\hat{\theta} \in argmin \; l_n(\theta)$

# SDG

On estime le theta de la log vraisemblance par $\nabla_{\theta} l_i(\theta)= -2(X_i-\theta)$

In [4]:
def SDG(theta_init, learn_rate, echantillon, n_iter):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*(-1)*gradient_log_vraisemblance(echantillon[i], theta_init)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*(-1)*gradient_log_vraisemblance(echantillon[i], theta)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*(-1)*gradient_log_vraisemblance(echantillon[i], theta)
            i+=1
            compteur+=1

    else:
        return theta

In [5]:
def SDG_IAWE(theta_init, learn_rate, n_iter, A, b, echantillon, k=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*importance_sampling_gradientlogvraisemblance(k=k, theta=theta_init, A=A, b=b, x=echantillon[i])

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*importance_sampling_gradientlogvraisemblance(k=k, theta=theta, A=A, b=b, x=echantillon[i])
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*importance_sampling_gradientlogvraisemblance(k=k, theta=theta, A=A, b=b, x=echantillon[i])
            i+=1
            compteur+=1

    else:
        return theta

In [6]:
def SDG_SUMO(theta_init, learn_rate, n_iter, A, b, echantillon, l=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*estimateur_SUMO_gradientlogvraisemblance(theta=theta_init, A=A, b=b, x=echantillon[i], r=0.6, l=l)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*estimateur_SUMO_gradientlogvraisemblance(theta=theta, A=A, b=b, x=echantillon[i], r=0.6, l=l)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*estimateur_SUMO_gradientlogvraisemblance(theta=theta, A=A, b=b, x=echantillon[i], r=0.6, l=l)
            i+=1
            compteur+=1

    else:
        return theta

In [7]:
def SDG_RR(theta_init, learn_rate, n_iter, A, b, echantillon, l=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*estimateur_ML_RR_gradientlogvraisemblance(x=echantillon[i], theta=theta_init, A=A, b=b, r=0.6, l=l)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*estimateur_ML_RR_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*estimateur_ML_RR_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
            i+=1
            compteur+=1

    else:
        return theta

In [8]:
def SDG_SS(theta_init, learn_rate, n_iter, A, b, echantillon, l=6):
    #Step 1: mélanger l'échantillon
    i=0
    compteur=0
    np.random.shuffle(echantillon)
    theta=theta_init-learn_rate*estimateur_ML_SS_gradientlogvraisemblance(x=echantillon[i], theta=theta_init, A=A, b=b, r=0.6, l=l)

    i+=1
    compteur+=1
    #Tant qu'on n'atteint pas le nombre d'itérations fixé, on actualise theta
    while compteur<n_iter:
        # Si on a parcouru tout l'échantillon alors que le nombre d'itérations n'est pas atteint,
        # On remélange l'échantillon, on reparcourt l'échantillon en commençant par le début
        # Le compteur n'est pas réinitialisé
        if i==len(echantillon):
           i=0
           np.random.shuffle(echantillon)
           theta=theta-learn_rate*estimateur_ML_SS_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
           i+=1
           compteur+=1
        else:
            theta=theta-learn_rate*estimateur_ML_SS_gradientlogvraisemblance(x=echantillon[i], theta=theta, A=A, b=b, r=0.6, l=l)
            i+=1
            compteur+=1

    else:
        return theta

In [9]:
np.random.seed(878)
theta_sdg=SDG(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, echantillon=echantillon_x)
theta_iawe=SDG_IAWE(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, k=6)
theta_SUMO=SDG_SUMO(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=6)
theta_RR=SDG_RR(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=6)
theta_SS=SDG_SS(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=6)

print("Vraie valeur de theta: {}".format(np.around(theta, 2)))
print("Estimation de theta par descente de gradient stochastique useuelle: {}".format(np.around(theta_sdg, 2)))
print("Estimation de theta par SDG IWAE: {}".format(np.around(theta_iawe, 2)))
print("Estimation de theta par SDG SUMO: {}".format(np.around(theta_SUMO, 2)))
print("Estimation de theta par SDG RR: {}".format(np.around(theta_RR, 2)))
print("Estimation de theta par SDG SS: {}".format(np.around(theta_SS, 2)))

Vraie valeur de theta: [-0.3  -1.01 -0.57  0.53 -0.45 -1.65 -1.66  1.26 -0.5  -2.16  0.18 -0.7
 -0.77  0.53  0.12 -0.5   0.69 -0.35 -0.43  1.35]
Estimation de theta par descente de gradient stochastique useuelle: [-0.03 -1.04 -0.47  0.34 -0.31 -1.87 -1.52  1.08 -0.37 -1.68  0.29 -0.6
 -0.58  0.56  0.16 -0.41  0.55 -0.33 -0.33  1.33]
Estimation de theta par SDG IWAE: [ 0.43  1.9   0.95 -0.43  0.58  2.79  2.31 -1.19  0.73  2.83 -0.18  1.18
  1.31 -0.49 -0.02  0.94 -0.81  0.52  0.89 -1.75]
Estimation de theta par SDG SUMO: [ 0.36  1.99  0.27 -0.68  0.49  2.48  2.44 -0.47  0.76  2.46  0.1   1.11
  1.72 -0.22  0.42  0.59 -0.28  0.75  0.91 -2.04]
Estimation de theta par SDG RR: [-7.86  0.44 -0.76 -3.34 -0.93  1.5   1.48 -4.38 -0.88  2.56 -3.24 -0.83
 -0.58 -3.48 -2.52 -1.38 -3.53 -1.3  -1.18 -4.79]
Estimation de theta par SDG SS: [-23.44   3.57   2.96  -3.76  -0.11   7.28   9.04  -4.26   5.74   8.91
  -4.93   3.28   6.11  -3.8   -2.04   2.04  -3.52   0.14   5.32  -5.92]


In [10]:
def procedure_MC_theta(M, L, theta, A, b, n):
    biais_SDG_M={}
    biais_IWAE_M={}
    biais_SUMO_M={}
    biais_SS_M={}
    biais_RR_M={}

    var_SDG_M={}
    var_IWAE_M={}
    var_SUMO_M={}
    var_SS_M={}
    var_RR_M={}

    l=1
    while l<=L:
        print(l)
        m=1
        estimations_SDG_M_l=np.array([])
        estimations_IWAE_M_l=np.array([])
        estimations_SUMO_M_l=np.array([])
        estimations_SS_M_l=np.array([])
        estimations_RR_M_l=np.array([])


        while m<=M:
            echantillon_x=generer_nech_gaussien(n)
            theta_SDG=SDG(theta_init=np.array([0]*20), learn_rate=0.01, echantillon=echantillon_x, n_iter=100)
            theta_IAWE=SDG_IAWE(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, k=l)
            theta_SUMO=SDG_SUMO(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)
            theta_RR=SDG_RR(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)
            theta_SS=SDG_SS(theta_init=np.array([0]*20), learn_rate=0.01, n_iter=100, A=A, b=b, echantillon=echantillon_x, l=l)

            if m==1:
                estimations_SDG_M_l=np.append(estimations_SDG_M_l, theta_SDG)
                estimations_IWAE_M_l= np.append(estimations_IWAE_M_l, theta_IAWE)
                estimations_SUMO_M_l=np.append(estimations_SUMO_M_l, theta_SUMO)
                estimations_RR_M_l=np.append(estimations_RR_M_l, theta_RR)
                estimations_SS_M_l=np.append(estimations_SS_M_l, theta_SS)
            
            else:
                estimations_SDG_M_l=np.vstack((estimations_SDG_M_l, theta_SDG))
                estimations_IWAE_M_l= np.vstack((estimations_IWAE_M_l, theta_IAWE))
                estimations_SUMO_M_l=np.vstack((estimations_SUMO_M_l, theta_SUMO))
                estimations_RR_M_l=np.vstack((estimations_RR_M_l, theta_RR))
                estimations_SS_M_l=np.vstack((estimations_SS_M_l, theta_SS))
            m+=1

        biais_SDG_M_l=np.mean(estimations_SDG_M_l, axis=0)-theta
        biais_IWAE_M_l=np.mean(estimations_IWAE_M_l, axis=0)-theta
        biais_IWAE_M_l=np.mean(estimations_IWAE_M_l, axis=0)-theta
        biais_SUMO_M_l=np.mean(estimations_SUMO_M_l, axis=0)-theta
        biais_SS_M_l=np.mean(estimations_SS_M_l, axis=0)-theta
        biais_RR_M_l=np.mean(estimations_RR_M_l, axis=0)-theta

        squared_biais_SDG_M_l=norm(biais_SDG_M_l)**2
        squared_biais_IWAE_M_l=norm(biais_IWAE_M_l)**2
        squared_biais_SUMO_M_l=norm(biais_SUMO_M_l)**2
        squared_biais_SS_M_l=norm(biais_SS_M_l)**2
        squared_biais_RR_M_l=norm(biais_RR_M_l)**2

        var_SDG_M_l=np.mean(norm(estimations_SDG_M_l-np.mean(estimations_IWAE_M_l, axis=0))**2)
        var_IWAE_M_l=np.mean(norm(estimations_IWAE_M_l-np.mean(estimations_IWAE_M_l, axis=0))**2)
        var_SUMO_M_l=np.mean(norm(estimations_SUMO_M_l-np.mean(estimations_SUMO_M_l, axis=0))**2)
        var_SS_M_l=np.mean(norm(estimations_SS_M_l-np.mean(estimations_SS_M_l, axis=0))**2)
        var_RR_M_l=np.mean(norm(estimations_RR_M_l-np.mean(estimations_RR_M_l, axis=0))**2)
        
        biais_SDG_M[l]=squared_biais_SDG_M_l
        biais_IWAE_M[l]=squared_biais_IWAE_M_l
        biais_SUMO_M[l]=squared_biais_SUMO_M_l
        biais_SS_M[l]=squared_biais_SS_M_l
        biais_RR_M[l]=squared_biais_RR_M_l
        
        var_SDG_M[l]=var_SDG_M_l
        var_IWAE_M[l]=var_IWAE_M_l
        var_SUMO_M[l]=var_SUMO_M_l
        var_SS_M[l]=var_SS_M_l
        var_RR_M[l]=var_RR_M_l

        l+=1

    return biais_SDG_M, biais_IWAE_M, biais_SUMO_M, biais_SS_M, biais_RR_M, var_SDG_M, var_IWAE_M, var_SUMO_M, var_SS_M, var_RR_M

In [11]:
np.random.seed(8554)

biais_SDG_M_theta, biais_IWAE_M_theta, biais_SUMO_M_theta, biais_SS_M_theta, biais_RR_M_theta, var_SDG_M_theta, var_IWAE_M_theta, var_SUMO_M_theta, var_SS_M_theta, var_RR_M_theta = procedure_MC_theta(M=1000, 
                                                                                                                                                                                                        L=6, 
                                                                                                                                                                                                        theta=theta, 
                                                                                                                                                                                                        A=A, 
                                                                                                                                                                                                        b=b,
                                                                                                                                                                                                        n=100)

1
[6.53255681e-24 2.12238423e-23 2.19556309e-22 8.27893289e-22
 1.08493129e-21 1.19185641e-21 2.60219717e-21 3.05642053e-21
 3.12017640e-21 6.05177794e-21 9.10020484e-21 9.49651246e-21
 1.22318187e-20 1.25513761e-20 2.85014989e-20 2.89471448e-20
 6.40192789e-20 7.10256666e-20 1.33009598e-19 1.45270592e-19
 1.61059073e-19 2.12499851e-19 2.45008966e-19 3.23324449e-19
 4.71978490e-19 7.30189623e-19 1.21941138e-18 1.41363631e-18
 1.81813912e-18 2.32874861e-18 5.41515919e-18 1.20859892e-17]
[1.51747040e-29 2.23762339e-29 6.34549840e-29 7.00233668e-28
 1.32385432e-27 4.27266657e-26 2.16157174e-25 4.88726459e-24]
[2.62103585e-30 3.97432354e-28 3.42833107e-27 4.31090440e-27
 1.02532136e-26 1.53208243e-26 7.07056980e-23 1.41251837e-21]
[3.39504394e-20 6.95343846e-20 8.54894852e-20 2.54995881e-19
 3.81133341e-19 6.91020285e-19 1.17168676e-18 1.50828516e-18
 1.71726369e-18 1.81886761e-18 5.32604610e-18 9.84804375e-18
 1.43435713e-17 2.90618812e-17 7.50336864e-17 1.87720499e-15]
[2.46253906e-40 3.

c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:182: RuntimeWarning: invalid value encountered in true_divide
  return num/denom
c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:252: RuntimeWarning: invalid value encountered in true_divide
  IWAE_O=np.sum(np.array([w_i*z_i for (w_i,z_i) in zip(array_w_O,z_O_theta)]), axis=0)/np.sum(array_w_O)
c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:253: RuntimeWarning: invalid value encountered in true_divide
  IWAE_E=np.sum(np.array([w_i*z_i for (w_i,z_i) in zip(array_w_E,z_E_theta)]), axis=0)/np.sum(array_w_E)
c:\Users\yanis\OneDrive\Documents\Monte Carlo\Projet-Monte-Carlo\src\estimators.py:254: RuntimeWarning: invalid value encountered in true_divide
  IWAE_OUE=np.sum(np.array([w_i*z_i for (w_i,z_i) in zip(array_w,z_theta)]), axis=0)/np.sum(array_w)


[nan]
[nan]
[2.50613901e-26 3.06864205e-26 6.31219266e-26 9.89411874e-26
 3.53805008e-25 1.10036383e-24 1.17778672e-24 1.69743022e-24
 2.30962744e-24 2.44523977e-24 2.44741320e-24 4.23298320e-24
 6.00372575e-24 7.27066024e-24 9.07699182e-24 1.06113828e-23
 1.29198477e-23 1.29891938e-23 1.30169099e-23 1.32278219e-23
 2.07517366e-23 2.43641453e-23 3.87309700e-23 4.62191307e-23
 4.86528678e-23 5.00899538e-23 5.12040040e-23 6.28536680e-23
 6.53004813e-23 6.88284861e-23 8.01515435e-23 9.39974998e-23
 1.07616941e-22 1.11915452e-22 1.49016289e-22 1.71210217e-22
 1.82363841e-22 1.83960508e-22 1.97639659e-22 2.02659656e-22
 2.09333065e-22 2.53237920e-22 2.75957393e-22 2.91351048e-22
 2.93196719e-22 4.04425084e-22 4.58025288e-22 4.71948794e-22
 4.87678716e-22 5.01155572e-22 5.90133444e-22 5.91816196e-22
 6.50502422e-22 6.97268659e-22 7.76184271e-22 8.07043064e-22
 8.12877066e-22 1.08155864e-21 1.11691720e-21 1.12181530e-21
 1.25206834e-21 1.35156802e-21 1.39390589e-21 1.40487546e-21
 1.56068633e

KeyboardInterrupt: 

In [ ]:
sorted_biais_IWAE_M_theta = sorted(biais_IWAE_M_gradient.items())
sorted_biais_SUMO_M_gradient = sorted(biais_SUMO_M_gradient.items())
sorted_biais_SS_M_gradient = sorted(biais_SS_M_gradient.items())
sorted_biais_RR_M_gradient = sorted(biais_RR_M_gradient.items())

sorted_var_IWAE_M_gradient = sorted(var_IWAE_M_gradient.items())
sorted_var_SUMO_M_gradient = sorted(var_SUMO_M_gradient.items())
sorted_var_SS_M_gradient = sorted(var_SS_M_gradient.items())
sorted_var_RR_M_gradient = sorted(var_RR_M_gradient.items())

l, biais_IAWE_gradient = zip(*sorted_biais_IWAE_M_theta)
l, biais_SUMO_gradient = zip(*sorted_biais_SUMO_M_gradient)
l, biais_SS_gradient = zip(*sorted_biais_SS_M_gradient)
l, biais_RR_gradient = zip(*sorted_biais_RR_M_gradient)

l, var_IAWE_gradient = zip(*sorted_var_IWAE_M_gradient) 
l, var_SUMO_gradient = zip(*sorted_var_SUMO_M_gradient) 
l, var_SS_gradient = zip(*sorted_var_SS_M_gradient) 
l, var_RR_gradient = zip(*sorted_var_RR_M_gradient) 

fig, axs = plt.subplots(2,1, figsize=(10, 10))

axs[0].plot(list_c, biais_IAWE_gradient, 'b', label="Biais au carré de l'estimateur IWAE", color="blue")
axs[0].plot(list_c, biais_SUMO_gradient, 'b', label="Biais au carré de l'estimateur SUMO", color="red")
axs[0].plot(list_c, biais_SS_gradient, 'b', label="Biais au carré de l'estimateur ML-SS", color="orange")
axs[0].plot(list_c, biais_RR_gradient, 'b', label="Biais au carré de l'estimateur ML-RR", color="green")

axs[0].legend()

fig.suptitle('Biais au carré et variance des estimateurs du gradient de la log-vraisemblance, en fonction du coût computationel', fontsize=16)

axs[1].plot(list_c, var_IAWE_gradient, 'b', label="Variance de l'estimateur IAWE", color="blue")
axs[1].plot(list_c, var_SUMO_gradient, 'b', label="Variance de l'estimateur SUMO", color="red")
axs[1].plot(list_c, var_SS_gradient, 'b', label="Variance de l'estimateur ML-SS", color="orange")
axs[1].plot(list_c, var_RR_gradient, 'b', label="Variance de l'estimateur ML-RR", color="green")


axs[1].legend()

plt.show()